In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import time
import numpy as np
from numpy_generator import NumpyGenerator
from lstm import LSTM
from label_packet import generate_label_data, label_packets
from random_perturb import random_perturb
from pso import PSO
from lstm import Lstm
from utils import softmax, print_header, get_events
from perm_seq2seq_attention import PermSeq2seqAttention
import argparse
import copy

def get_args(jupyter_args = None):
    parser = argparse.ArgumentParser()

    parser.add_argument('--permute_truncated', required=False, action='store_true', help="")
    parser.add_argument('--use_prob_embedding', required=False, action='store_true', help="")
    parser.add_argument('--sequence_length', required=False, type=int, default=10, help="")
    parser.add_argument('--rv', required=False, type=int, default=1, help="")

    if jupyter_args is not None:
        args = parser.parse_args(jupyter_args)
    else: 
        args = parser.parse_args()

    return args

#jupyter_args = ['--permute_truncated', '--use_prob_embedding']
args = get_args()
seq2seq_config = {"sequence_length": args.sequence_length, 
                  "permute_truncated": args.permute_truncated,
                  "use_prob_embedding": args.use_prob_embedding,
                  "rv": args.rv
                  }   

# -----------get the preprocessed training and testing saved as .npy files
test_label_infection = np.load('preprocessed/test_label_infection.npy')
train_label_infection = np.load('preprocessed/train_label_infection.npy')
test_data_infection = np.load('preprocessed/test_data_infection.npy')
train_data_infection = np.load('preprocessed/train_data_infection.npy')

test_label_reconnaissance = np.load('preprocessed/test_label_reconnaissance.npy')
train_label_reconnaissance = np.load('preprocessed/train_label_reconnaissance.npy')
test_data_reconnaissance = np.load('preprocessed/test_data_reconnaissance.npy')
train_data_reconnaissance = np.load('preprocessed/train_data_reconnaissance.npy')

test_label_attack = np.load('preprocessed/test_label_attack.npy')
train_label_attack = np.load('preprocessed/train_label_attack.npy')
test_data_attack = np.load('preprocessed/test_data_attack.npy')
train_data_attack = np.load('preprocessed/train_data_attack.npy')




ModuleNotFoundError: No module named 'data_preprocess'

In [7]:
attack_detector = Lstm("lstm-attack")

In [8]:
attack_detector.name

'lstm-attack'

In [9]:
attack_detector = Lstm("lstm-attack")
reconnaissance_detector = Lstm("lstm-recon")
infection_detector = Lstm("lstm-infection")

for detector in [attack_detector, reconnaissance_detector, infection_detector]:
    #train data
    train_data = all_data['train']
    train_examples = train_data[0]
    train_labels = train_data[1]

    #test data
    test_data = all_data['test']
    test_examples = test_data[0]
    test_labels = test_data[1]
        
    features_len = train_examples.shape[1]
    print('features len is ', features_len)
    
    print_header("Training {} detector".format(detector.name))
    detector.learning(features_len, train_examples, train_labels)
                    
    print_header("Measureing {} detector performance on test data".format(detector.name))
    detector.detection(test_examples, test_labels)


features len is  41
------------------------
Training attack detector
------------------------


2023-04-30 18:38:42.335963: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-30 18:38:42.336777: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-30 18:38:42.589488: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 715.34GiB/s
2023-04-30 18:38:42.589629: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/oracle/instantclient_19_8:/opt/oracle/instantclient_19_8:/p/cuda-11.5
2023-04-30 18:38:42.589709: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 

the info of dataset, (19152, 1, 41)
shuffle is enabled for training
Epoch 1/50
539/539 - 5s - loss: 0.1415 - accuracy: 0.9517 - auc: 0.9885 - recall: 0.9582 - precision: 0.9455 - val_loss: 0.0558 - val_accuracy: 0.9796 - val_auc: 0.9982 - val_recall: 0.9889 - val_precision: 0.9721
Epoch 2/50
539/539 - 1s - loss: 0.0673 - accuracy: 0.9745 - auc: 0.9963 - recall: 0.9863 - precision: 0.9635 - val_loss: 0.0413 - val_accuracy: 0.9849 - val_auc: 0.9989 - val_recall: 0.9899 - val_precision: 0.9809
Epoch 3/50
539/539 - 1s - loss: 0.0585 - accuracy: 0.9784 - auc: 0.9970 - recall: 0.9879 - precision: 0.9694 - val_loss: 0.0485 - val_accuracy: 0.9849 - val_auc: 0.9982 - val_recall: 0.9899 - val_precision: 0.9809
Epoch 4/50
539/539 - 1s - loss: 0.0552 - accuracy: 0.9807 - auc: 0.9974 - recall: 0.9908 - precision: 0.9711 - val_loss: 0.0372 - val_accuracy: 0.9885 - val_auc: 0.9991 - val_recall: 0.9970 - val_precision: 0.9811
Epoch 5/50
539/539 - 1s - loss: 0.0524 - accuracy: 0.9804 - auc: 0.9977 - re

KeyError: 'train'

In [5]:
train_events_per_detector = []
for detector in  [attack_detector, infection_detector, reconnaissance_detector]:
    #examples_train = infection_unb_data['train'][0]
    examples_train = all_data['infection']['train'][0]
    train_events = detector.predict(examples_train)
    print('proportion of events tagged by {} detector: '.format(detector.get_name()), np.sum(np.array(train_events)>0.5)/len(train_events))
    train_events_per_detector.append(train_events)

benign_events = []
no_events = len(train_events_per_detector[0])
for i  in range(no_events):
    not_attack = 1-train_events_per_detector[0][i]
    not_infection = 1-train_events_per_detector[1][i]
    not_recon = 1-train_events_per_detector[2][i]
    benign_events.append(not_attack * not_infection * not_recon)
train_events_per_detector.append(benign_events)
print('proportion of events estimated to be benign : ', np.sum(np.array(benign_events)>0.5)/len(benign_events))
train_events_per_detector = np.array(train_events_per_detector).squeeze()


proportion of events tagged by lstm-attack detector:  0.03945907760621657
proportion of events tagged by lstm-infection detector:  0.4250681198910082
proportion of events tagged by lstm-recon detector:  0.47739428802099104
proportion of events estimated to be benign :  0.07008779897063276


In [6]:
def softmax(x):
    """
    Compute the softmax of vector x.
    """
    # Subtract the maximum value of x from each element of x to avoid overflow
    # and make the softmax numerically stable.
    z = x - np.max(x)
    
    # Calculate the exponential of each element in z.
    exp_z = np.exp(z)
    
    # Calculate the sum of the exponential values.
    sum_exp_z = np.sum(exp_z)
    
    # Divide each element of the exponential vector by the sum of exponential values.
    softmax_z = exp_z / sum_exp_z
    
    return softmax_z

train_events_softmax = []
for i in range(no_events):
    softmax_event = softmax(train_events_per_detector[:,i])
    train_events_softmax.append(softmax_event)

In [7]:
np.sum(np.argmax(train_events_softmax, axis=-1)==0)/no_events

0.029468160258350994

In [8]:
train_events_softmax

[array([0.16987948, 0.46177995, 0.19846092, 0.1698796 ], dtype=float32),
 array([0.1748777 , 0.1748777 , 0.1748777 , 0.47536692], dtype=float32),
 array([0.36583766, 0.36134997, 0.13638298, 0.13642937], dtype=float32),
 array([0.1752403 , 0.47305506, 0.1752429 , 0.17646174], dtype=float32),
 array([0.17725915, 0.44604293, 0.18591627, 0.19078164], dtype=float32),
 array([0.17676704, 0.47281808, 0.17478679, 0.17562805], dtype=float32),
 array([0.1748777 , 0.1748777 , 0.47536692, 0.1748777 ], dtype=float32),
 array([0.17726189, 0.2245559 , 0.38976145, 0.20842077], dtype=float32),
 array([0.17726189, 0.2245559 , 0.38976145, 0.20842077], dtype=float32),
 array([0.17726189, 0.2245559 , 0.38976145, 0.20842077], dtype=float32),
 array([0.17487796, 0.17487796, 0.4753653 , 0.17487884], dtype=float32),
 array([0.17726189, 0.2245559 , 0.38976145, 0.20842077], dtype=float32),
 array([0.17726189, 0.2245559 , 0.38976145, 0.20842077], dtype=float32),
 array([0.17726189, 0.2245559 , 0.38976145, 0.20842

In [9]:
from perm_seq2seq_attention import PermSeq2seqAttention

In [10]:
seq2seq = PermSeq2seqAttention('seq2seq')

In [11]:
train_events_softmax[4]

array([0.17725915, 0.44604293, 0.18591627, 0.19078164], dtype=float32)

In [12]:
config = {"sequence_length": 10, 
          "permute_truncated": True,
          "use_prob_embedding": False,
          "rv": 1
        }

In [13]:
train_label_infection_unb.sum()

107.0

In [14]:
#seq2seq.learning(train_events_softmax, train_label_infection_unb, config)
seq2seq.learning(train_events_softmax, all_data['infection']['train'][1], config)

Permute truncated is True
Use prob embedding is False
Permute Truncated is enabled
truncated idxs are [[ 0  1  2  3  4  5  6  7  8  9]
 [ 1  2  3  4  5  6  7  8  9 10]
 [ 2  3  4  5  6  7  8  9 10 11]
 [ 3  4  5  6  7  8  9 10 11 12]
 [ 4  5  6  7  8  9 10 11 12 13]
 [ 5  6  7  8  9 10 11 12 13 14]
 [ 6  7  8  9 10 11 12 13 14 15]
 [ 7  8  9 10 11 12 13 14 15 16]
 [ 8  9 10 11 12 13 14 15 16 17]
 [ 9 10 11 12 13 14 15 16 17 18]
 [10 11 12 13 14 15 16 17 18 19]
 [11 12 13 14 15 16 17 18 19 20]
 [12 13 14 15 16 17 18 19 20 21]
 [13 14 15 16 17 18 19 20 21 22]
 [14 15 16 17 18 19 20 21 22 23]
 [15 16 17 18 19 20 21 22 23 24]
 [16 17 18 19 20 21 22 23 24 25]
 [17 18 19 20 21 22 23 24 25 26]
 [18 19 20 21 22 23 24 25 26 27]
 [19 20 21 22 23 24 25 26 27 28]]
perm truncated idxs are [[ 0  1  5  3  2  4  9  7  8  6]
 [ 1  5  4  2  3  6  7 10  9  8]
 [ 4  2  5  3  7  8 11 10  6  9]
 [ 4  3  5 12  9 11 10  8  7  6]
 [ 4  5 12  6 11  8  9  7 10 13]
 [ 5  9  7 11 12  8  6 10 13 14]
 [ 8  6 11 12  

In [15]:
res_probs, res_idxs = seq2seq.analysis(train_events_softmax, all_data['infection']['train'][1], config)

Permute truncated is True
Use prob embedding is False


In [16]:
train_events_per_detector = []
for detector in  [attack_detector, infection_detector, reconnaissance_detector]:
    #examples_train = infection_unb_data['train'][0]
    examples_train = all_data['infection']['train'][0]
    train_events = detector.predict(examples_train)
    print('proportion of events tagged by {} detector: '.format(detector.get_name()), np.sum(np.array(train_events)>0.5)/len(train_events))
    train_events_per_detector.append(train_events)

benign_events = []
no_events = len(train_events_per_detector[0])
for i  in range(no_events):
    not_attack = 1-train_events_per_detector[0][i]
    not_infection = 1-train_events_per_detector[1][i]
    not_recon = 1-train_events_per_detector[2][i]
    benign_events.append(not_attack * not_infection * not_recon)
train_events_per_detector.append(benign_events)
print('proportion of events estimated to be benign : ', np.sum(np.array(benign_events)>0.5)/len(benign_events))
train_events_per_detector = np.array(train_events_per_detector).squeeze()

proportion of events tagged by lstm-attack detector:  0.03945907760621657
proportion of events tagged by lstm-infection detector:  0.4250681198910082
proportion of events tagged by lstm-recon detector:  0.47739428802099104
proportion of events estimated to be benign :  0.07008779897063276


In [17]:
iPS_pred = infection_detector.predict(all_data['infection']['train'][0])


In [18]:
infection_tagged_PS = []

for idx, pred in enumerate(iPS_pred):
    if pred>0.5:
        infection_tagged_PS.append(idx)


In [19]:
retrain_pos = []
retrain_neg = []
for idx in infection_tagged_PS:
    if idx in res_idxs:
        retrain_pos.append(idx)
    else:
        retrain_neg.append(idx)

In [20]:
retrain_labels = [0]*len(retrain_neg) + [1]*len(retrain_pos)
retrain_labels = np.array(retrain_labels)
retrain_data = all_data['infection']['train'][0][retrain_neg].tolist() + all_data['infection']['train'][0][retrain_pos].tolist()
retrain_data = np.array(retrain_data)

In [21]:
import copy
retrain_labels = copy.deepcopy(all_data['infection']['train'][1])
retrain_data = copy.deepcopy(all_data['infection']['train'][0])

for idx, l in enumerate(retrain_labels):
    if idx in retrain_pos:
        retrain_labels[idx] = 1
    if idx in retrain_neg:
        retrain_labels[idx] = 0


In [22]:
len(retrain_pos)

8359

In [23]:
np.sum(all_data['infection']['train'][1])

9909.0

In [24]:
np.sum(retrain_labels)

9995.0

In [25]:
def retrain_detector(detector, retrain_data, retrain_labels, test_data, test_labels):

    features_len = retrain_data.shape[1]
    print('features len is ', features_len)
    
    print_header("Retraining {} detector".format('infection'))
    detector.learning(features_len, retrain_data, retrain_labels)
                    
    print_header("Measureing {} detector performance on test data".format('infection'))
    detector.detection(test_data, test_labels)
    return detector 

In [26]:
len(retrain_labels)

19818

In [27]:
retrain_labels.shape

(19818,)

In [28]:
test_data =  all_data['infection']['test'][0]
test_labels =  all_data['infection']['test'][1]
train_data = all_data['infection']['train'][0]

infection_detector = retrain_detector(infection_detector, train_data, retrain_labels, test_data, test_labels)

features len is  41
-----------------------------
Retraining infection detector
-----------------------------
the info of dataset, (19818, 1, 41)
shuffle is enabled for training
Epoch 1/50
558/558 - 3s - loss: 0.3631 - accuracy: 0.8613 - auc_4: 0.9047 - recall_4: 0.8170 - precision_4: 0.8994 - val_loss: 0.2948 - val_accuracy: 0.8930 - val_auc_4: 0.9299 - val_recall_4: 0.8270 - val_precision_4: 0.9495
Epoch 2/50
558/558 - 1s - loss: 0.2726 - accuracy: 0.8962 - auc_4: 0.9350 - recall_4: 0.8292 - precision_4: 0.9599 - val_loss: 0.2778 - val_accuracy: 0.8946 - val_auc_4: 0.9348 - val_recall_4: 0.8280 - val_precision_4: 0.9518
Epoch 3/50
558/558 - 1s - loss: 0.2607 - accuracy: 0.8993 - auc_4: 0.9375 - recall_4: 0.8289 - precision_4: 0.9673 - val_loss: 0.2648 - val_accuracy: 0.8961 - val_auc_4: 0.9370 - val_recall_4: 0.8280 - val_precision_4: 0.9551
Epoch 4/50
558/558 - 1s - loss: 0.2542 - accuracy: 0.9021 - auc_4: 0.9378 - recall_4: 0.8312 - precision_4: 0.9709 - val_loss: 0.2664 - val_accu

In [29]:
infection_detector_new = Lstm("lstm-infection")

558/558 - 1s - loss: 0.2375 - accuracy: 0.9074 - auc_2: 0.9413 - recall_2: 0.8315 - precision_2: 0.9813 - val_loss: 0.2272 - val_accuracy: 0.9178 - val_auc_2: 0.9477 - val_recall_2: 0.8476 - val_precision_2: 0.9768
Epoch 50/50
558/558 - 1s - loss: 0.2349 - accuracy: 0.9083 - auc_2: 0.9421 - recall_2: 0.8319 - precision_2: 0.9830 - val_loss: 0.2284 - val_accuracy: 0.9173 - val_auc_2: 0.9468 - val_recall_2: 0.8476 - val_precision_2: 0.9756
classifier is generated with time step 1
------------------------------------------------------
Measureing infection detector performance on test data
------------------------------------------------------
accuracy: 0.9534609024332624 precision: 0.8201058201058201 recall: 0.48742138364779874 f1: 0.6114398422090731 AUC: 0.7429572600142976
fp: 34 ,tp: 155 ,fn: 163 ,tn: 3881




558/558 - 1s - loss: 0.2450 - accuracy: 0.9041 - auc_4: 0.9409 - recall_4: 0.8330 - precision_4: 0.9734 - val_loss: 0.2513 - val_accuracy: 0.9011 - val_auc_4: 0.9416 - val_recall_4: 0.8240 - val_precision_4: 0.9710
Epoch 10/50
...
Measureing infection detector performance on test data
------------------------------------------------------
accuracy: 0.9560595322466336 precision: 0.83 recall: 0.5220125786163522 f1: 0.6409266409266409 AUC: 0.6131448950577121
fp: 34 ,tp: 166 ,fn: 152 ,tn: 3881



558/558 - 1s - loss: 0.2220 - accuracy: 0.9129 - auc_5: 0.9450 - recall_5: 0.8345 - precision_5: 0.9911 - val_loss: 0.2125 - val_accuracy: 0.9183 - val_auc_5: 0.9492 - val_recall_5: 0.8462 - val_precision_5: 0.9931
classifier is generated with time step 1
------------------------------------------------------
Measureing infection detector performance on test data
------------------------------------------------------
accuracy: 0.9553508150248051 precision: 0.8086124401913876 recall: 0.5314465408805031 f1: 0.6413662239089183 AUC: 0.6137051495216752
fp: 40 ,tp: 169 ,fn: 149 ,tn: 3875


In [30]:
infection_detector_new = Lstm("lstm-infection")
infection_detector = retrain_detector(infection_detector, train_data, retrain_labels, test_data, test_labels)

features len is  41
-----------------------------
Retraining infection detector
-----------------------------
the info of dataset, (19818, 1, 41)
shuffle is enabled for training
Epoch 1/50
558/558 - 4s - loss: 0.3629 - accuracy: 0.8624 - auc_5: 0.9046 - recall_5: 0.8175 - precision_5: 0.9001 - val_loss: 0.2818 - val_accuracy: 0.8951 - val_auc_5: 0.9312 - val_recall_5: 0.8442 - val_precision_5: 0.9448
Epoch 2/50
558/558 - 1s - loss: 0.2770 - accuracy: 0.8928 - auc_5: 0.9325 - recall_5: 0.8247 - precision_5: 0.9564 - val_loss: 0.2537 - val_accuracy: 0.9041 - val_auc_5: 0.9404 - val_recall_5: 0.8452 - val_precision_5: 0.9629
Epoch 3/50
558/558 - 1s - loss: 0.2655 - accuracy: 0.8978 - auc_5: 0.9350 - recall_5: 0.8279 - precision_5: 0.9642 - val_loss: 0.2542 - val_accuracy: 0.9051 - val_auc_5: 0.9415 - val_recall_5: 0.8412 - val_precision_5: 0.9693
Epoch 4/50
558/558 - 1s - loss: 0.2603 - accuracy: 0.9003 - auc_5: 0.9379 - recall_5: 0.8292 - precision_5: 0.9683 - val_loss: 0.2525 - val_accu

In [5]:
len(all_data['reconnaissance']['train'][1])

10016

In [7]:
np.sum(all_data['reconnaissance']['train'][1]==0)

2811

In [8]:
np.sum(all_data['reconnaissance']['train'][1]==1)/len(all_data['reconnaissance']['train'][0])

0.7193490415335463

In [9]:
np.sum(all_data['attack']['train'][1]==1)/len(all_data['attack']['train'][0])

0.0439297124600639

In [10]:
np.sum(all_data['infection']['train'][1]==1)/len(all_data['infection']['train'][0])

0.010682907348242811